In [1]:
#import numpy as np
import xarray
import rasterio
import pandas as pd

In [2]:
mw_path = r"../Data/microwave-rs/CumJour-North-ssmi-1979-2022-H19.nc"
mw_out_path = r'../Data/microwave-rs/mw-tif/'

In [3]:
dataset_mw = xarray.open_dataset(mw_path).rio.write_crs(
        "epsg:3413", inplace=True)

In [7]:
datelist = pd.date_range(start="2019-07-01",end="2019-07-31")

In [8]:
for i in datelist:
    print(i)
    dataset_day = dataset_mw.where(dataset_mw["time"] == i, drop=True)
    data_mw_day = dataset_day["melt"]
    #print(data_mw_day)
    data_mw_day.rio.to_raster(f"{mw_out_path}{i.date()}_mw.tif") # 



2019-07-01 00:00:00
2019-07-02 00:00:00
2019-07-03 00:00:00
2019-07-04 00:00:00
2019-07-05 00:00:00
2019-07-06 00:00:00
2019-07-07 00:00:00
2019-07-08 00:00:00
2019-07-09 00:00:00
2019-07-10 00:00:00
2019-07-11 00:00:00
2019-07-12 00:00:00
2019-07-13 00:00:00
2019-07-14 00:00:00
2019-07-15 00:00:00
2019-07-16 00:00:00
2019-07-17 00:00:00
2019-07-18 00:00:00
2019-07-19 00:00:00
2019-07-20 00:00:00
2019-07-21 00:00:00
2019-07-22 00:00:00
2019-07-23 00:00:00
2019-07-24 00:00:00
2019-07-25 00:00:00
2019-07-26 00:00:00
2019-07-27 00:00:00
2019-07-28 00:00:00
2019-07-29 00:00:00
2019-07-30 00:00:00
2019-07-31 00:00:00
